**Автор:** Сергей Шмаков

**Обновление этого документа** и не только публикуется на канале в Телеграм - https://telegram.me/smm_python

Другие мои площадки:
* ВКонтакте - https://vk.com/smmblog
* Youtube - https://youtube.com/smmblog
* Блог - http://freesmm.ru

# Урок 7: Анализ аудитории сообщества, сайта, списка ID 

**запустите код ниже** (видео ютуб)

In [2]:
from IPython.display import YouTubeVideo
YouTubeVideo("y-VaNbaQ51A")

В 5 уроке мы анализировали стену сообщества (количество лайков\репостов\комментариев), а в 6 научились собирать статистику группы, использую access_token. Всё это не зря, так как полученные знания и опыт из предыдущих уроков нам полностью понадобится, чтобы в итоге получить это:

![alt](https://api.monosnap.com/rpc/file/download?id=FdFXicUSPvVaIsSiGxGTLMHofdX2uw)<br>
![alt](https://api.monosnap.com/rpc/file/download?id=HoFXgiMyEcmMjjGjf7UY4GCu3CWlne)

Вообще зачем нужен анализ аудитории сообщества, сайта, списка ID:
* хотим знать какие группы пользователей являются подписчиками сообщества \ посетители сайта
* выстроить на основе полученных данных предположения, которые могут повлиять на то как вести в целом взаимодействие с аудиторией

Есть несколько способов анализа аудитории сообщества\аудитории сайта (при установленном коде ретаргетинга):
1. Если есть доступ к статистике, то мы можем посмотреть часть информации, которую отображает ВКонтакте
2. Спарсить всех пользователей со списком всех полей - https://vk.com/dev/groups.getMembers
3, Используя рекламное API - https://vk.com/dev/ads

## Минусы и Плюсы

### Через статистику
**Плюсы**:

+ быстро
+ есть история развития сообщества

**Минусы** :

+ если статистика закрыта, то ничего проанализировать не получится, либо придётся писать админу, чтобы он открыл статистику
+ мало данных

### Через парсинг участников сообщества
**Плюсы**:

+ можно получить информацию о каждом участнике
+ есть история развития сообщества

**Минусы** :

+ не все данные можно получить, так как пользователи заполняют не все поля (возраст, город, страна и тд)
+ сложность обработки данных (собранный файл будет содержать много данных (если сообщество большое) и работать в Excel будет с ним очень сложно\медленно)

### Через рекламное API
**Плюсы**:

+ множество параметров (больше чем можно получить через статистику группы или через парсинг участников)
+ можно анализировать аудиторию сообщества, сайта (при наличии кода ретаргетинга и доступа к нему), приложения

**Минусы** :

+ будем знать только количество аудитории
+ количество аудитории чаще всего не совпадает с реальным, но погрешность приемлемая
![alt](https://api.monosnap.com/rpc/file/download?id=HCgYyn5LEZyhbIC2n2i6TjBHhmQp6V)
> Конечно, можно добавить еще анализ действий на стене и другие варианты, но остановимся пока на этих 3х.


Теперь когда у нас есть 3 варианта анализа давайте поставим задачу, чтобы выбрать "правильный" метод или несколько

**Задача**

> + узнать как распределены подписчики сообщества https://vk.com/smmblog по **полу** и **возрасту** (с **точностью** до года - сколько 20-летних М и Ж и тд) - от 20 до 25 лет
+ сколько пользователей использовал голоса в приложениях ВКонтакте (по каждой половозрастной группе)
+ какими **устройствами** они пользуются

Теперь вернёмся к возможным методам анализа. 

* Парсинг участников из-за своей сложности нам не подойдёт, так как нам не нужно знать "кто" конкретно + мы не можем получить данные об использовании голосов и какими устройствами они пользуются (конечно можно, но для этого придётся парсить их стену на предмет публикации через приложения c iOS или Android.

* Анализ аудитории через статистику тоже не даст нам те данные, которые нужны. Плюс в статистике сообщества мы можем получить данные только о половозрастных группах
![alt](https://api.monosnap.com/rpc/file/download?id=CcEf33gLi1uJzMnZ9ja3qzgNOeUZej)

Конечно, я поставил задачу так, чтоб 3 метод анализа был наиболее подходящим для решения. Этот метод можно использовать и руками, меня параметры при создании/обновлении объявления. Так как изучаем Python, то руками мы будем только писать код, который позволит нам анализировать аудиторию гораздо быстрее и по множеству параметров.

Преступим к написанию кода - последовательно усложняя код.

# Пишем код
Для начала нам нужно правильно сформировать запрос к методу https://vk.com/dev/ads.getTargetingStats
> у меня на это ушло пару дней, несколько вопросов, пару взрывов мозга, но всё привело к тому, что вы читаете этот готовый урок.

Вся сложность была в правильном осмыслении "criteria - сериализованный JSON-объект, описывающий параметры таргетинга. Описание объекта criteria см. ниже. "
![alt](https://api.monosnap.com/rpc/file/download?id=1y6lH4hYpqbGyumlV4W8G7NNemNNim)

## Шаг 1: отправляем запрос

Но для начала обновите токен с получением прав на рекламу!!!

In [44]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json

Token = "b526251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog

#Формируем запрос

r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                     params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                            'criteria':json.dumps({'sex':0,
                                                    'age_from':20, #возраст "от"
                                                    'age_to':20, #возраст "до"
                                                    'groups':groups
                                                   }),
                             #'ad_id':23301483,
                             'ad_format':1, #может влиять на стоимость cpc и cpm
                              'link_url':'http://yoursite.cоm',
                              'link_domain':'http://yoursite.cоm',
                             'v':'5.52',
                             'access_token':Token})

r.json()

{'response': {'audience_count': 572,
  'recommended_cpc': '34.69',
  'recommended_cpm': '10.09'}}

**Что мы получили?** 
Количество пользователей(**573**), которые подходят под параметры аудитории: 
* пол - любой
* возраст - от 20 до 20
* участники сообщества - https://vk.com/smmblog

**Теория**

Что делает json.dumps
* http://pythonworld.ru/moduli/modul-json.html
> json.dumps(obj, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, default=None, sort_keys=False, **kw) - сериализует obj в строку JSON-формата.


In [2]:
import json
a = json.dumps({'sex':0,
            'age_from':20,
            'age_to':20,
            'groups':1
            })
a

'{"groups": 1, "age_from": 20, "age_to": 20, "sex": 0}'

Как видите, получилась строка (ограничена одинарными кавычками слева и справа), а кавычки внутри словаря стали двойными.

## Шаг 2: получаем данные о нескольких группах возрастов

В первом шаге мы получили данные только для 20-20, но нам нужно получить и к 21-21, и т.д. Как быть? Использовать **цикл for**, а чтобы было по чему проходить циклу, то нужно создать список с возрастом:

```python
age_from = [20,21,22,23,24,25] #возраст "от"
```

In [4]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json

Token = "b526251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog

age_from = [20,21,22,23,24,25]


for age_from in age_from:
    #Формируем запрос
    r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                         params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                'criteria':json.dumps({'sex':0,
                                                        'age_from':age_from, #возраст "от" - заменили
                                                        'age_to':20, #возраст "до"
                                                        'groups':groups
                                                       }),
                                 #'ad_id':23301483,
                                 'ad_format':1, #может влиять на стоимость cpc и cpm
                                  'link_url':'http://yoursite.cоm',
                                  'link_domain':'http://yoursite.cоm',
                                 'v':'5.52',
                                 'access_token':Token})

    r.json()

In [5]:
r.json()

{'error': {'error_code': 9,
  'error_msg': 'Flood control',
  'request_params': [{'key': 'oauth', 'value': '1'},
   {'key': 'method', 'value': 'ads.getTargetingStats'},
   {'key': 'account_id', 'value': '1600169068'},
   {'key': 'v', 'value': '5.52'},
   {'key': 'link_url', 'value': 'http://yoursite.cоm'},
   {'key': 'criteria',
    'value': '{"groups": 38369814, "age_from": 25, "age_to": 20, "sex": 0}'},
   {'key': 'ad_format', 'value': '1'},
   {'key': 'link_domain', 'value': 'http://yoursite.cоm'}]}}

## Шаг 3: устранение ошибки

Так, выдало ошибк с кодом **9**. Давайте разбираться что это за ошибка (список всех ошибок с кодами - https://vk.com/dev/errors)
![alt](https://api.monosnap.com/rpc/file/download?id=i0VQqlNyuzvLnsS15qwk7wFWjY0gl1)

Так, слишком много однотипных действий. Мы помним, что ограничения обращений к api vk составляет 3 в секунду. То есть нам нужно добавить *time.sleep(.33)*

In [6]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time

Token = "b526251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog

age_from = [20,21,22,23,24,25]


for age_from in age_from:
    #Формируем запрос
    r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                         params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                'criteria':json.dumps({'sex':0,
                                                        'age_from':age_from, #возраст "от" - заменили
                                                        'age_to':20, #возраст "до"
                                                        'groups':groups
                                                       }),
                                 #'ad_id':23301483,
                                 'ad_format':1, #может влиять на стоимость cpc и cpm
                                  'link_url':'http://yoursite.cоm',
                                  'link_domain':'http://yoursite.cоm',
                                 'v':'5.52',
                                 'access_token':Token})

    r.json()
    time.sleep(.33)

In [7]:
r.json()

{'error': {'error_code': 9,
  'error_msg': 'Flood control',
  'request_params': [{'key': 'oauth', 'value': '1'},
   {'key': 'method', 'value': 'ads.getTargetingStats'},
   {'key': 'account_id', 'value': '1600169068'},
   {'key': 'v', 'value': '5.52'},
   {'key': 'link_url', 'value': 'http://yoursite.cоm'},
   {'key': 'criteria',
    'value': '{"groups": 38369814, "age_from": 25, "age_to": 20, "sex": 0}'},
   {'key': 'ad_format', 'value': '1'},
   {'key': 'link_domain', 'value': 'http://yoursite.cоm'}]}}

Так, ничего не изменилось. Лезем в рекламный api и ищем информацию. Может мы что-то упустили... https://vk.com/dev/ads
![alt](https://api.monosnap.com/rpc/file/download?id=w3XHRQCOqg7zlv5sIfAwnY0GFhFW0P)

Ага. Давайте посмотрим что там - https://vk.com/dev/ads_limits

### Ограничения на вызов методов Ads API
При работе с Ads API стоит учитывать следующие особенности платформы: 

* **Примитивное ограничение** — ограничение на количество вызываемых методов в течение короткого промежутка времени для каждого пользователя.
* **Часовое ограничение** — ограничение на количество вызываемых методов в течение часа для каждого рекламного кабинета

#### Примитивное ограничение

На серверах стоят различные ограничения на частоту вызовов методов. В частности, платформа не позволяет Вам запускать методы более **двух раз** в течение **одной** секунды.

> То есть как минимум time.sleep(.5) , но читаем дальше!

#### Конкретные цифры
Примитивное ограничение :

* **"Сложные"** методы (ads.createAds, ads.updateAds, **ads.getTargetingStats**) разрешается запускать не чаще 5 раз в 10 секунд. 
* Все методы (включая "сложные") разрешается запускать не чаще 2 раз в секунду. 
* ads.getUploadURL разрешается запускать со стандартной частотой для вызовов методов API.

> Так как мы используем метод **ads.getTargetingStats**, то мы его можем запрашивать не чаще 5 раз в 10 секунд или 1 каждые 2 секунды. То есть
```python
time.sleep(2)
```

Попробуем снова


In [ ]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time

Token = "251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog

age_from = [20,21,22,23,24,25]


for age_from in age_from: #будет подставляться сначала 20,21,22 и тд.
    #Формируем запрос
    r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                         params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                'criteria':json.dumps({'sex':0,
                                                        'age_from':age_from, #возраст "от" - заменили
                                                        'age_to':20, #возраст "до"
                                                        'groups':groups
                                                       }),
                                 #'ad_id':23301483,
                                 'ad_format':1, #может влиять на стоимость cpc и cpm
                                  'link_url':'http://yoursite.cоm',
                                  'link_domain':'http://yoursite.cоm',
                                 'v':'5.52',
                                 'access_token':Token})

    print (r.json())
    time.sleep(2)

Этот код у нас не сработал. Почему? Наверное потому, что **age_to** мы не меняем. То есть, если для 20-20 в первом прохождении цикла мы еще получим какое-то значение, то при втором когда будет "возраст от 21 до 20" ничего не произойдёт. age_to не может быть меньше age_from.

Давайте сделаем **age_to = 25**

In [11]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time

Token = "1234251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog

age_from = [20,21,22,23,24,25]


for age_from in age_from: #будет подставляться сначала 20,21,22 и тд.
    #Формируем запрос
    r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                         params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                'criteria':json.dumps({'sex':0,
                                                        'age_from':age_from, #возраст "от" - заменили
                                                        'age_to':25, #возраст "до"
                                                        'groups':groups
                                                       }),
                                 #'ad_id':23301483,
                                 'ad_format':1, #может влиять на стоимость cpc и cpm
                                  'link_url':'http://yoursite.cоm',
                                  'link_domain':'http://yoursite.cоm',
                                 'v':'5.52',
                                 'access_token':Token})

    print (r.json())
    time.sleep(2)

{'response': {'recommended_cpc': '35.15', 'audience_count': 7036, 'recommended_cpm': '10.09'}}
{'response': {'recommended_cpc': '38.86', 'audience_count': 6400, 'recommended_cpm': '10.95'}}
{'response': {'recommended_cpc': '38.86', 'audience_count': 5552, 'recommended_cpm': '11.41'}}
{'response': {'recommended_cpc': '39.19', 'audience_count': 4497, 'recommended_cpm': '11.20'}}
{'response': {'recommended_cpc': '39.48', 'audience_count': 3280, 'recommended_cpm': '11.42'}}
{'response': {'recommended_cpc': '41.29', 'audience_count': 1698, 'recommended_cpm': '12.01'}}


Ура! Что-то получили. Теперь давайте разберём что получили.

* Первое прохождение цикла: age_from = 20, age_to 25 (от 20 до 25, любого пола) = 'audience_count': 7036
* Второе прохождение цилка: age_from = 21, age_to 25 (от 21 до 25, любого пола) = 'audience_count': 6400
* Третье прохождение цилка: age_from = 22, age_to 25 (от 21 до 25, любого пола) = 'audience_count': 4497
* и тд

Надеюсь логика понятна. Результат всё-таки не тот, что нам нужен. Напомню, нам нужно:
> + узнать как распределены подписчики сообщества https://vk.com/smmblog по **полу** и **возрасту** (с **точностью** до года - сколько 20-летних М и Ж и тд) - от 20 до 25 лет
+ сколько пользователей использовал голоса в приложениях ВКонтакте (по каждой половозрастной группе)
+ какими **устройствами** они пользуются


Чего не хватает?
* мы получили данные только для *любого пола*
* данные для сегмента лет-лет получили только для 25-25 - 'audience_count': 1698

Какие выводы можно сделать?
* нужно использовать несколько циклов (разный пол, разный возраст - как минимум 2)
* возраст от-до должен быть одинаковый

## Шаг 4: дорабатываем код - еще цикл for

In [12]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time

Token = "32134526251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog

age_from = [20,21,22,23,24,25]
sex = [1,2]

for sex in sex:
    for age_from in age_from: #будет подставляться сначала 20,21,22 и тд.
        age_to = age_from #лет "от" = лет "до" 20=20 и тд.
        #Формируем запрос
        r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                             params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                    'criteria':json.dumps({'sex':sex, #подставляем 1 или 2 (Ж-1 или М-2 или Все-0)
                                                            'age_from':age_from, #возраст "от" - заменили
                                                            'age_to':age_to, #возраст "до" - заменили
                                                            'groups':groups
                                                           }),
                                     #'ad_id':23301483,
                                     'ad_format':1, #может влиять на стоимость cpc и cpm
                                      'link_url':'http://yoursite.cоm',
                                      'link_domain':'http://yoursite.cоm',
                                     'v':'5.52',
                                     'access_token':Token})

        print (r.json())
        time.sleep(2)

{'response': {'recommended_cpc': '30.98', 'audience_count': 200, 'recommended_cpm': '9.01'}}
{'response': {'recommended_cpc': '33.04', 'audience_count': 315, 'recommended_cpm': '9.38'}}
{'response': {'recommended_cpc': '33.21', 'audience_count': 421, 'recommended_cpm': '9.78'}}
{'response': {'recommended_cpc': '34.69', 'audience_count': 534, 'recommended_cpm': '10.09'}}
{'response': {'recommended_cpc': '34.69', 'audience_count': 628, 'recommended_cpm': '10.09'}}
{'response': {'recommended_cpc': '39.23', 'audience_count': 699, 'recommended_cpm': '11.42'}}


TypeError: 'int' object is not iterable

Ошибка! В чем причина? Давайте посмотрим на циклы и какие переменные они используют:

```python
age_from = [20,21,22,23,24,25]
sex = [1,2]

for sex in sex:
    for age_from in age_from: #будет подставляться сначала 20,21,22 и тд.
```

Как будет проходить цикл:
1. sex = 1, age_from = 20
2. sex = 1, age_from = 21
3. sex = 1, age_from = 22
4. sex = 1, age_from = 23
5. sex = 1, age_from = 24
6. sex = 1, age_from = 25 ###После 6 прохода мы получаем ошибку. Причина в том, что у нас кончились элементы в списке
7. sex = 2, age_from = ???

На 7 шаге для нас понятно, то нужно подставить снова 20, но как это поправить в коде? 

> Прежде чем смотреть ответ ниже - **подумайте самостоятельно**.



In [ ]:
#думай сам

In [ ]:
#думай сам

In [13]:
#думай сам

Надеюсь, вы самостоятельно подумали, так как это лучше помогает разобраться. Я сам не с первого раза написал правильный вариант, переставляю определенные переменные\циклы и тд.

Как же исправить?

Смотрите, переменная **age_from** у нас объявлена вне цикла. Нам же, как я писал выше, нужно после смены *sex=1* на *sex=2* (первый цикл) снова пойти по значениям возрастов в списке *age_from = [20,21,22,23,24,25]*

Переменную age_from нужно поместить внутрь цикла **for sex in sex**:

**Было**
```python
age_from = [20,21,22,23,24,25]
sex = [1,2]
for sex in sex:
    for age_from in age_from: #будет подставляться сначала 20,21,22 и тд.
```
**Стало**
```python
sex = [1,2]
for sex in sex:
    age_from = [20,21,22,23,24,25] #переместили в цикл
    for age_from in age_from: #будет подставляться сначала 20,21,22 и тд.
```

In [15]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time

Token = "2134b526251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog


sex = [1,2]

for sex in sex:
    age_from = [20,21,22,23,24,25] #переместили в цикл
    for age_from in age_from: #будет подставляться сначала 20,21,22 и тд.
        age_to = age_from #лет "от" = лет "до" 20=20 и тд.
        #Формируем запрос
        r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                             params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                    'criteria':json.dumps({'sex':sex, #подставляем 1 или 2 (Ж-1 или М-2 или Все-0)
                                                            'age_from':age_from, #возраст "от" - заменили
                                                            'age_to':age_to, #возраст "до" - заменили
                                                            'groups':groups
                                                           }),
                                     #'ad_id':23301483,
                                     'ad_format':1, #может влиять на стоимость cpc и cpm
                                      'link_url':'http://yoursite.cоm',
                                      'link_domain':'http://yoursite.cоm',
                                     'v':'5.52',
                                     'access_token':Token})

        print (r.json())
        time.sleep(2)

{'response': {'recommended_cpc': '30.98', 'audience_count': 200, 'recommended_cpm': '9.01'}}
{'response': {'recommended_cpc': '33.04', 'audience_count': 315, 'recommended_cpm': '9.38'}}
{'response': {'recommended_cpc': '33.21', 'audience_count': 421, 'recommended_cpm': '9.78'}}
{'response': {'recommended_cpc': '34.69', 'audience_count': 534, 'recommended_cpm': '10.09'}}
{'response': {'recommended_cpc': '34.69', 'audience_count': 628, 'recommended_cpm': '10.09'}}
{'response': {'recommended_cpc': '39.23', 'audience_count': 699, 'recommended_cpm': '11.42'}}
{'response': {'recommended_cpc': '34.69', 'audience_count': 373, 'recommended_cpm': '10.09'}}
{'response': {'recommended_cpc': '36.09', 'audience_count': 450, 'recommended_cpm': '10.20'}}
{'response': {'recommended_cpc': '37.83', 'audience_count': 543, 'recommended_cpm': '10.83'}}
{'response': {'recommended_cpc': '39.97', 'audience_count': 567, 'recommended_cpm': '11.62'}}
{'response': {'recommended_cpc': '39.35', 'audience_count': 809

Поздравляю! Уже ближе к тому, что нам нужно.

Мы получили 12 результатов (2 пола * 6 возрастов). Давайте удостоверимся, что мы получаем точно данные для 20-20,21-21 и т.д. для нужного пола.

Доработаем **print**

```python
print (sex, age_from, age_to, r.json())
```

In [16]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time

Token = "12346251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog


sex = [1,2]

for sex in sex:
    age_from = [20,21,22,23,24,25] #переместили в цикл
    for age_from in age_from: #будет подставляться сначала 20,21,22 и тд.
        age_to = age_from #лет "от" = лет "до" 20=20 и тд.
        #Формируем запрос
        r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                             params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                    'criteria':json.dumps({'sex':sex, #подставляем 1 или 2 (Ж-1 или М-2 или Все-0)
                                                            'age_from':age_from, #возраст "от" - заменили
                                                            'age_to':age_to, #возраст "до" - заменили
                                                            'groups':groups
                                                           }),
                                     #'ad_id':23301483,
                                     'ad_format':1, #может влиять на стоимость cpc и cpm
                                      'link_url':'http://yoursite.cоm',
                                      'link_domain':'http://yoursite.cоm',
                                     'v':'5.52',
                                     'access_token':Token})

        print (sex, age_from, age_to, r.json())
        time.sleep(2)

1 20 20 {'response': {'recommended_cpc': '30.98', 'audience_count': 200, 'recommended_cpm': '9.01'}}
1 21 21 {'response': {'recommended_cpc': '33.04', 'audience_count': 315, 'recommended_cpm': '9.38'}}
1 22 22 {'response': {'recommended_cpc': '33.21', 'audience_count': 421, 'recommended_cpm': '9.78'}}
1 23 23 {'response': {'recommended_cpc': '34.69', 'audience_count': 534, 'recommended_cpm': '10.09'}}
1 24 24 {'response': {'recommended_cpc': '34.69', 'audience_count': 628, 'recommended_cpm': '10.09'}}
1 25 25 {'response': {'recommended_cpc': '39.23', 'audience_count': 699, 'recommended_cpm': '11.42'}}
2 20 20 {'response': {'recommended_cpc': '34.69', 'audience_count': 373, 'recommended_cpm': '10.09'}}
2 21 21 {'response': {'recommended_cpc': '36.09', 'audience_count': 450, 'recommended_cpm': '10.20'}}
2 22 22 {'response': {'recommended_cpc': '37.83', 'audience_count': 543, 'recommended_cpm': '10.83'}}
2 23 23 {'response': {'recommended_cpc': '39.97', 'audience_count': 567, 'recommended

## Шаг 6: оставляем только нужные данные

Из 4 урока мы научились оставлять только те данные, которые нам нужны. Так как из *{'response': {'recommended_cpc': '41.29', 'audience_count': 999, 'recommended_cpm': '12.01'}}* , нам нужно только значение для **'audience_count': 999**

In [18]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time

Token = "1234b526251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog


sex = [1,2]

for sex in sex:
    age_from = [20,21,22,23,24,25] #переместили в цикл
    for age_from in age_from: #будет подставляться сначала 20,21,22 и тд.
        age_to = age_from #лет "от" = лет "до" 20=20 и тд.
        #Формируем запрос
        r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                             params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                    'criteria':json.dumps({'sex':sex, #подставляем 1 или 2 (Ж-1 или М-2 или Все-0)
                                                            'age_from':age_from, #возраст "от" - заменили
                                                            'age_to':age_to, #возраст "до" - заменили
                                                            'groups':groups
                                                           }),
                                     #'ad_id':23301483,
                                     'ad_format':1, #может влиять на стоимость cpc и cpm
                                      'link_url':'http://yoursite.cоm',
                                      'link_domain':'http://yoursite.cоm',
                                     'v':'5.52',
                                     'access_token':Token})
        
        response = r.json()
        audience_count = response['response']['audience_count']
        

        print (sex, age_from, age_to, audience_count)
        time.sleep(2)

1 20 20 200
1 21 21 315
1 22 22 421
1 23 23 534
1 24 24 628
1 25 25 699
2 20 20 373
2 21 21 450
2 22 22 543
2 23 23 567
2 24 24 809
2 25 25 999


Так, теперь нам бы это записать в csv, чтобы руками не копировать

## Шаг 7: записываем полученные данные в csv
Этому мы научились в 6 уроке

In [19]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time
import csv

Token = "123426251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog


sex = [1,2]

for sex in sex:
    age_from = [20,21,22,23,24,25] #переместили в цикл
    for age_from in age_from: #будет подставляться сначала 20,21,22 и тд.
        age_to = age_from #лет "от" = лет "до" 20=20 и тд.
        #Формируем запрос
        r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                             params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                    'criteria':json.dumps({'sex':sex, #подставляем 1 или 2 (Ж-1 или М-2 или Все-0)
                                                            'age_from':age_from, #возраст "от" - заменили
                                                            'age_to':age_to, #возраст "до" - заменили
                                                            'groups':groups
                                                           }),
                                     #'ad_id':23301483,
                                     'ad_format':1, #может влиять на стоимость cpc и cpm
                                      'link_url':'http://yoursite.cоm',
                                      'link_domain':'http://yoursite.cоm',
                                     'v':'5.52',
                                     'access_token':Token})
        
        response = r.json()
        audience_count = response['response']['audience_count']
        #записываем данные в csv
        with open('groups_analytics.csv', 'a', newline='') as csvfile:
            datawriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
            datawriter.writerow([groups]+[sex]+[age_from]+[age_to]+[audience_count])
        
        #на всякий случай выводим на экран, чтобы видеть что всё ок.
        print (sex, age_from, age_to, audience_count)
        time.sleep(2)

1 20 20 200
1 21 21 315
1 22 22 421
1 23 23 534
1 24 24 628
1 25 25 699
2 20 20 373
2 21 21 450
2 22 22 543
2 23 23 567
2 24 24 809
2 25 25 999


В csv данные мы успешно записали. Об этом говорит наличие файла в папке
![alt](https://api.monosnap.com/rpc/file/download?id=XDKSRuhWSHQxPR4gyfeSUZXXqLwvAV)

Что внутри?
![alt](https://api.monosnap.com/rpc/file/download?id=TNE6pn6HvAFLSNL3DKi0w23t8NvEjK)

Колонки:
1. - это ID сообщества (мало ли будем сразу несколько анализировать)
2. - пол
3. - возраст от
4. - возраст до
5. - размер аудитории 

Конечно, мы можем руками доработать таблица до более читаемого вида
![alt](https://api.monosnap.com/rpc/file/download?id=RfKgOw9041EOGCUUnvJ3D9fjloDUqo)





По этим данным мы уже без проблем можем строить графики, диаграмы и проводить анализ. В Экселе это удобнее делать, чем в OpenOffice, поэтому сохраняю в эксель-формат и работаю в нём.

После некоторых манипуляций с графиками получил следующее:
![alt](https://api.monosnap.com/rpc/file/download?id=QYSIXY4UDOy2eShcr0OrbaZp7IyyNr)



Так как с единицами и двойками сложно работать, то давайте заменим 1 на "Ж", а 2 на "М" (ctrl+h в экселе, предварительно выделив диапазон для замены)
![alt](https://api.monosnap.com/rpc/file/download?id=R4C8tQT0f7n2WgzKkfezFpX42Sue3w)



В итоге получаем
![alt](https://api.monosnap.com/rpc/file/download?id=azqcE7jQtbQ2CNc49HrJszZnSAp2yu)

Картина конечно неполная, так как возраст всего от 20 до 25, но уже можно пару слов сказать и сделать выводы, какая дополнительная информация нужна:
* чем взрослее аудитория (от 20 до 25) тем больше её в сообществе, что говорит о зрелости сообщества
* М-аудитория(20-25) = 3741 - две трети от этого составляет м-25 (999) + м-24 (809)
* Ж-аудитория(20-25) = 2797
* М/Ж-аудитория растёт вместе с ростом возраста 
![alt](https://api.monosnap.com/rpc/file/download?id=epcEhkC5h19cQgbmAOe1atVP2xP974)<br>
![alt](https://api.monosnap.com/rpc/file/download?id=MHAfU3FltmULwnWBIJtFviQjosAp5f)<br>



Чего не хватает? Наверное отношение к общему количеству участников, чтобы иметь представление о размере той или иной половозрастной группы в рамках сообщества. 

Сейчас у меня в сообществе 26 989 пользователей (14 мая 2016). Нужно создать новый столбец в Экселе и вписать в него формулу "=(**аудитория-половозрастной**-**группы/аудитория сообщества**)\*100" - 
![alt](https://api.monosnap.com/rpc/file/download?id=nnqVvNqcRiJyduul1NP1oeFkrZUQ9F)

Помните!!! Количество аудитории, получаемое через рекламное API меньше, чем реально.

Вот какие данные отдаёт если запросить: любой пол и возраст "от 0 до 99"
```python
{'response': {'audience_count': 23674,
  'recommended_cpc': '40.34',
  'recommended_cpm': '11.73'}}
```

Поэтому какое значение подставлять для расчёта размера сегмента выбирать вам. Я записал оба (мало ли)


![alt](https://api.monosnap.com/rpc/file/download?id=riKrMw9lgQpKol9j8ehcNFrvpU4cOT)


In [25]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time
import csv

Token = "31234526251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog


for sex in range(1,3): #пройдёт по значениям 1 и 2 (3 не включительно!!!)
    for age_from in range(100): #будет подставляться сначала 0,1,2,3 и тд.
        age_to = age_from #лет "от" = лет "до" 20=20 и тд.
        #Формируем запрос
        r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                             params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                    'criteria':json.dumps({'sex':sex, #подставляем 1 или 2 (Ж-1 или М-2 или Все-0)
                                                            'age_from':age_from, #возраст "от" - заменили
                                                            'age_to':age_to, #возраст "до" - заменили
                                                            'groups':groups
                                                           }),
                                     #'ad_id':23301483,
                                     'ad_format':1, #может влиять на стоимость cpc и cpm
                                      'link_url':'http://yoursite.cоm',
                                      'link_domain':'http://yoursite.cоm',
                                     'v':'5.52',
                                     'access_token':Token})
        
        response = r.json()
        audience_count = response['response']['audience_count']
        #записываем данные в csv
        with open('groups_analytics.csv', 'a', newline='') as csvfile:
            datawriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
            datawriter.writerow([groups]+[sex]+[age_from]+[age_to]+[audience_count])
        
        #на всякий случай выводим на экран, чтобы видеть что всё ок.
        print (sex, age_from, age_to, audience_count)
        time.sleep(2)

2 0 0 13916
2 1 1 0
2 2 2 0
2 3 3 0
2 4 4 0
2 5 5 0
2 6 6 0
2 7 7 0
2 8 8 0
2 9 9 0
2 10 10 0
2 11 11 0
2 12 12 0
2 13 13 27
2 14 14 75
2 15 15 117
2 16 16 131
2 17 17 140
2 18 18 178
2 19 19 237
2 20 20 372
2 21 21 444
2 22 22 544
2 23 23 566
2 24 24 807
2 25 25 1002
2 26 26 970
2 27 27 1005
2 28 28 933
2 29 29 801
2 30 30 663
2 31 31 556
2 32 32 407
2 33 33 334
2 34 34 254
2 35 35 213
2 36 36 219
2 37 37 120
2 38 38 120
2 39 39 86
2 40 40 91
2 41 41 78
2 42 42 59
2 43 43 49
2 44 44 52
2 45 45 50
2 46 46 35
2 47 47 32
2 48 48 21
2 49 49 23
2 50 50 22
2 51 51 25
2 52 52 10
2 53 53 11
2 54 54 10
2 55 55 10
2 56 56 13
2 57 57 4
2 58 58 8
2 59 59 8
2 60 60 6
2 61 61 5
2 62 62 4
2 63 63 1
2 64 64 4
2 65 65 3
2 66 66 7
2 67 67 1
2 68 68 2
2 69 69 4
2 70 70 7
2 71 71 5
2 72 72 1
2 73 73 2
2 74 74 2
2 75 75 3
2 76 76 4
2 77 77 1
2 78 78 1
2 79 79 2
2 80 80 1
2 81 81 2
2 82 82 4
2 83 83 0
2 84 84 1
2 85 85 0
2 86 86 2
2 87 87 1
2 88 88 3
2 89 89 2
2 90 90 2
2 91 91 3
2 92 92 5
2 93 93 3
2 94 9

Если собрать данные для всех возрастов, то получим в итоге такой график
![alt](https://api.monosnap.com/rpc/file/download?id=xtFM2gH1uO7jjh2rZsATMp6bjjjYoj)

Интерактивный график вы можете посмотреть здесь (Google Docs): https://docs.google.com/spreadsheets/d/1g9HUQKYyZ2tdadh8KX8ZDzRO_cq14o9KUCEMMYxoGWw/pubchart?oid=48159470&format=interactive

Удалил строки где значения были нулевыми либо близкими к нулю. В итоге получилось от 13 до 55 лет.

Давайте попробуем собрать данные для группы возраста с 13 до 55 с шагом 3 (13-16, 16-19 и тд). Для этого в цикле **for** укажем от какого числа считать, до какого и с каким шагом для переменной **age_from**... 
```python
for age_from in range(от, до, шаг)
```
>**Теория** - про циклы http://pythonworld.ru/osnovy/cikly-for-i-while-operatory-break-i-continue-volshebnoe-slovo-else.html

а переменная **age_to** будет равна "возрасту от + 3 года"
```python
age_to = age_from + 3
```

In [28]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time
import csv

Token = "31234526251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog


for sex in range(1,3):
    for age_from in range(13,56,3): #будет подставляться сначала 13,16,18 и тд.
        age_to = age_from + 3 #лет "от" = лет "до" 13+3, 16+3 и тд.
        #Формируем запрос
        r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                             params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                    'criteria':json.dumps({'sex':sex, #подставляем 1 или 2 (Ж-1 или М-2 или Все-0)
                                                            'age_from':age_from, #возраст "от" - заменили
                                                            'age_to':age_to, #возраст "до" - заменили
                                                            'groups':groups
                                                           }),
                                     #'ad_id':23301483,
                                     'ad_format':1, #может влиять на стоимость cpc и cpm
                                      'link_url':'http://yoursite.cоm',
                                      'link_domain':'http://yoursite.cоm',
                                     'v':'5.52',
                                     'access_token':Token})
        
        response = r.json()
        audience_count = response['response']['audience_count']
        #записываем данные в csv
        with open('groups_analytics.csv', 'a', newline='') as csvfile:
            datawriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
            datawriter.writerow([groups]+[sex]+[age_from]+[age_to]+[audience_count])
        
        #на всякий случай выводим на экран, чтобы видеть что всё ок.
        print (sex, age_from, age_to, audience_count)
        time.sleep(2)

1 13 16 296
1 16 19 423
1 19 22 1149
1 22 25 2457
1 25 28 3095
1 28 31 2511
1 31 34 1485
1 34 37 840
1 37 40 456
1 40 43 278
1 43 46 196
1 46 49 132
1 49 52 88
1 52 55 87
1 55 58 52
2 13 16 373
2 16 19 737
2 19 22 1719
2 22 25 3098
2 25 28 4180
2 28 31 3147
2 31 34 1647
2 34 37 865
2 37 40 440
2 40 43 294
2 43 46 202
2 46 49 121
2 49 52 86
2 52 55 45
2 55 58 36


Обработав данные в экселе, можем построить нормированную линейчатую диаграмму с накоплением
![alt](https://api.monosnap.com/rpc/file/download?id=447DAFpYJ2eAPpd7mFxvNRMBXaR8t1)





Сравнивая одинаковые группы возрастов видно, что мужская аудитория чаще всего занимает больше 50%. Чем старше становится аудитория, тем меньше разница между мужской и женской аудиторией.

Незначительный перевес женской аудитории перед мужской есть в группе возрастов **37-40**. Потом женская аудитория занимает больше 50% с возрастной группы **46-49**.

Давайте сравним это с тем, что нам отдаёт статистика группы по "уникам" и по "Охвату" по половозрастным группам (за всё время)
![alt](https://api.monosnap.com/rpc/file/download?id=5ScqTz2xDmTrplEerIXFcNPZOosoQS)
![alt](https://api.monosnap.com/rpc/file/download?id=fZjD3gIyTffs1YExoTRKxNwfg9qhEK)




Так как у нас тут гистограмма без накопления, то такую же сделаем и в Экселе

![alt](https://api.monosnap.com/rpc/file/download?id=mh73YBNjYaK69QkYS9YpCiR1I8bers)<br>

Чтобы возрастные группы совпадали нужно начинать с 15

```python
for age_from range(15,55,3)
```

Строить выводы на основе полученых данных не для этого урока. Нам важно скорее сначала данные собрать по первичному заданию, а потому уже приступать к анализу и выводам.

> **Обновление задания**
Давайте теперь попробуем получить те же параметры ауитории, но только для жителей Москвы и Санкт-Петербурга.

## Шаг 8: еще один цикл и идентификаторы городов
Обратимся снова к методу https://vk.com/dev/ads.getTargetingStats и ищем информацию о городах
![alt](https://api.monosnap.com/rpc/file/download?id=8E9VOhksp9gEl9UtJlSvqqoMw1jcXE)<br>

От нас требуют передачу списка городов в виде списка чисел. То же самое мы указывали для пола, где 0 - все, 1 - Ж, 2 - М. 

Где взять идентификаторы городов?
Есть специальный набор методов Данные ВК - https://vk.com/dev/database, с помощью которых можно узнать ID нужных объектов (страна, город, регионы, список школ и тд)

Нам нужен https://vk.com/dev/database.getCities, который возвращает список городов. Так как нам нужно узнать всего 2 ID (Москва , Санкт-Петербург), то будем запускать запрос прямо в ВК (снизу у каждого метода есть форма)
![alt](https://api.monosnap.com/rpc/file/download?id=unwk56U9FcWDbtzwNaclYDzbokOF22)<br>



* Москва = 1
* Санкт-Петербург = 2

Теперь нам нужно добавить еще один цикл for и записать данные аудитории в 2 новых столбца

In [30]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time
import csv

Token = "123451b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен
groups = 38369814 #id vk.com/smmblog


for sex in range(1,3):
    for age_from in range(13,56,3): #будет подставляться сначала 13,16,18 и тд.
        age_to = age_from + 3 #лет "от" = лет "до" 13+3, 16+3 и тд.
        cities = [1,2] #1 - Москва, 2 - Санкт-Петербург
        for cities in cities:
            #Формируем запрос
            r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                                 params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                        'criteria':json.dumps({'sex':sex, #подставляем 1 или 2 (Ж-1 или М-2 или Все-0)
                                                                'age_from':age_from, #возраст "от" - заменили
                                                                'age_to':age_to, #возраст "до" - заменили
                                                                'groups':groups,
                                                                'cities':cities
                                                               }),
                                         #'ad_id':23301483,
                                         'ad_format':1, #может влиять на стоимость cpc и cpm
                                          'link_url':'http://yoursite.cоm',
                                          'link_domain':'http://yoursite.cоm',
                                         'v':'5.52',
                                         'access_token':Token})

            response = r.json()
            audience_count = response['response']['audience_count']
            #записываем данные в csv
            with open('groups_analytics.csv', 'a', newline='') as csvfile:
                datawriter = csv.writer(csvfile, delimiter=',',
                                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
                datawriter.writerow([groups]+[sex]+[cities]+[age_from]+[age_to]+[audience_count])

            #на всякий случай выводим на экран, чтобы видеть что всё ок.
            print (sex, cities, age_from, age_to, audience_count)
            time.sleep(2)

1 1 13 16 22
1 2 13 16 11
1 1 16 19 37
1 2 16 19 34
1 1 19 22 175
1 2 19 22 170
1 1 22 25 439
1 2 22 25 353
1 1 25 28 547
1 2 25 28 476
1 1 28 31 470
1 2 28 31 384
1 1 31 34 254
1 2 31 34 237
1 1 34 37 124
1 2 34 37 145
1 1 37 40 61
1 2 37 40 76
1 1 40 43 38
1 2 40 43 36
1 1 43 46 23
1 2 43 46 31
1 1 46 49 17
1 2 46 49 12
1 1 49 52 8
1 2 49 52 18
1 1 52 55 9
1 2 52 55 11
1 1 55 58 7
1 2 55 58 3
2 1 13 16 32
2 2 13 16 21
2 1 16 19 100
2 2 16 19 40
2 1 19 22 243
2 2 19 22 138
2 1 22 25 532
2 2 22 25 287
2 1 25 28 840
2 2 25 28 493
2 1 28 31 629
2 2 28 31 428
2 1 31 34 311
2 2 31 34 250
2 1 34 37 167
2 2 34 37 119
2 1 37 40 93
2 2 37 40 53
2 1 40 43 62
2 2 40 43 26
2 1 43 46 40
2 2 43 46 19
2 1 46 49 17
2 2 46 49 18
2 1 49 52 9
2 2 49 52 15
2 1 52 55 9
2 2 52 55 7
2 1 55 58 3
2 2 55 58 5


После небольшой обработки данных (сортировка по ID города) получилась следующая таблица.
![alt](https://api.monosnap.com/rpc/file/download?id=Pp1Wc1DZrlCEgwxKdGY6bPVq4iy9Pg)<br>



Так как я не особый мастер работы данными в Экселе, то буду переносить данные в новые столбцы с добавлением форматирования. На выходе получили такую компактную таблицу
![alt](https://api.monosnap.com/rpc/file/download?id=HXgPOwJDZQxqChEDy8JrpqYgzoUDio)<br>

Если добавить условного форматирования для каждого столбца. В моём примере будет отмечаться от наименьшего (зеленым) до наибольшего (красным) по каждому столбцы
![alt](https://api.monosnap.com/rpc/file/download?id=5uYOQNOhysqp90gYbNSarsnZPPiK6J)


В итоге у нас такая вот разноцветная таблица, которую визуально уже проще анализировать
![alt](https://api.monosnap.com/rpc/file/download?id=rBXyJjKdX6y1QcauwRcOAekwYDAbnN)

Легко увидеть какая возрастная группа составляет наибольшую аудитория по каждому полу и городу - **25-28**.
![alt](https://api.monosnap.com/rpc/file/download?id=NoRMY4G9ju6MFaRdHDP36FxPHidiy6)

Попытался построить графики по каждому полу
![alt](https://api.monosnap.com/rpc/file/download?id=HoFXgiMyEcmMjjGjf7UY4GCu3CWlne)

Разница между количетвом московской и питерской женской аудитории меньше, чем мужская. По некоторым возрастным группам она составляет почти 2 раза.



## Логика кода

Надеюсь, вы примерно поняли как нужно формировать запрос и как формировать циклы, чтобы получить необходимые данные для анализа

1. Определяем по каким параметрам мы хотим перебирать: пол, возраст, город
2. Сколько параметров перебора будет, столько должно быть циклов for
3. Параметры мы записыем тут
```python
'criteria':json.dumps({'sex':sex, #(Ж-1 или М-2 или Все-0)
                        'age_from':age_from, #возраст "от" - заменили
                        'age_to':age_to, #возраст "до" - заменили
                        'groups':groups,
                        'cities':cities,
                        'параметр':'какое-то значение' #переменная, значение (целое число, 'строка', дробное число)
                       }
```

4. Не забываем писать в CSV новые столбцы при переборе (см. пример для города)


## Анализ аудитории ретаргетинга

Примера анализа аудитории сайта (http://freesmm.ru), на котором установлен код ретаргетинга. Это может быть и не аудитория сайта, а загруженный вами список ID в ретаргетинг.
![alt](https://api.monosnap.com/rpc/file/download?id=obqbFhc7dUGf9netL2inGDIFu5joIk)<br>

Перебор будем делать только по группе возрастов, полу - 2 параметра = 2 цикла. Также потребуется передать дополнительный параметр\критерий **retargeting_groups**
![alt](https://api.monosnap.com/rpc/file/download?id=EYkPPqwkUlsoeZiTKBACYe820Wcnkn)

Чтобы получить список ID кодов ретаргетинга воспользуемся этим методом https://vk.com/dev/ads.getTargetGroups прямо из интерфейса (как и для городов)
![alt](https://api.monosnap.com/rpc/file/download?id=QRUwFQXGu8LA8A01wgzH5K6nIKQHeY)


In [31]:
import requests #для отправки запроса и обработки json
import json  #поребуется для сериализации json
import time
import csv

Token = "1234b526251b9f8aa7eb0c0595913591749b3164d6874a04bce379a8a9050549fae7b6038c13a2bd9" #подставьте свой токен

for sex in range(1,3):
    for age_from in range(15,56,3): #будет подставляться сначала 13,16,18 и тд.
        age_to = age_from + 3 #лет "от" = лет "до" 13+3, 16+3 и тд.

        #Формируем запрос
        r = requests.get('https://api.vk.com/method/ads.getTargetingStats',
                             params={'account_id':1600169068, #id рекламного аккаунта https://vk.com/ads?act=settings
                                    'criteria':json.dumps({'sex':sex, #подставляем 1 или 2 (Ж-1 или М-2 или Все-0)
                                                            'age_from':age_from, #возраст "от" - заменили
                                                            'age_to':age_to, #возраст "до" - заменили
                                                            'retargeting_groups':159794 #id кода ретаргетинга на сайте
                                                           }),
                                     #'ad_id':23301483,
                                     'ad_format':1, #может влиять на стоимость cpc и cpm
                                      'link_url':'http://yoursite.cоm',
                                      'link_domain':'http://yoursite.cоm',
                                     'v':'5.52',
                                     'access_token':Token})

        response = r.json()
        audience_count = response['response']['audience_count']
        #записываем данные в csv
        with open('site_analytics.csv', 'a', newline='') as csvfile:
            datawriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
            datawriter.writerow([sex]+[age_from]+[age_to]+[audience_count])

        #на всякий случай выводим на экран, чтобы видеть что всё ок.
        print (sex, cities, age_from, age_to, audience_count)
        time.sleep(2)

1 2 15 18 8972
1 2 18 21 13169
1 2 21 24 18812
1 2 24 27 22403
1 2 27 30 19784
1 2 30 33 12296
1 2 33 36 6574
1 2 36 39 3548
1 2 39 42 2032
1 2 42 45 1282
1 2 45 48 918
1 2 48 51 632
1 2 51 54 510
1 2 54 57 428
2 2 15 18 12908
2 2 18 21 22657
2 2 21 24 27618
2 2 24 27 31165
2 2 27 30 25357
2 2 30 33 14993
2 2 33 36 8423
2 2 36 39 4793
2 2 39 42 2676
2 2 42 45 1668
2 2 45 48 1095
2 2 48 51 664
2 2 51 54 502
2 2 54 57 416


В итоге получилось следующее
![alt](https://api.monosnap.com/rpc/file/download?id=FdFXicUSPvVaIsSiGxGTLMHofdX2uw)<br>

Тут анализировался код, который установлен аж с прошлого года, поэтому эти данные нам особо ничего не скажут. Если хотите делать какие-то выводы, то вам нужно устанавливать на сайт код ретаргетинга, который будет удалять аудиторию спустя какое-то время
![alt](https://api.monosnap.com/rpc/file/download?id=8LpM0WWKQLGXuattPBTnt0aArp4jKt)<br>

и потом регулярно (раз в неделю\месяц\6 месяцев) запускать скрипт анализа аудитории по необходимым параметрам. Эти данные будет интереснее анализировать, так как у вас будет представление о каждому периоде + если будете вести учёт событий (публикация поста\реклама в директе и тд), то анализ данных может привести к интересным выводам.


# Итоги 7 урока

**Python-часть**
1. Разобрали как сделать сериализованный json
2. Научились запускать много циклов

**Аналитика**
1. Разобрали как можно проанализировать сообщество, список сообществ, аудиторию ретаргетинга по множеству параметров, которые предоставляет рекламное API
2. Как можно обработать данные в экселе
3. Как можно анализировать аудиторию сайта 

## Домашнее задание
1. Получить токен для работы с рекламным API. Если не знаете как, то 6 урок вам в помощь **иначе на запустятся примеры из этого урока**
2. Проанализировать 2 сообщества с близкой тематикой по полу, возрасту
3. Проанализировать аудиторию группы ретаргетинга (загрузка по ID, сайта) по полу, возрасту и устройствам (iphone, androide и тд) - https://vk.com/dev/ads.getSuggestions?params[section]=user_devices&params[ids]=0&params[v]=5.52

Ответы и вопросы по домашке публикуйте здесь - https://vk.com/topic-38369814_33529150

**используйте поле ниже для решения самостоятельных задач**

-----

Далее: [Урок 8: Как открыть\закрыть стену ВКонтакте по таймеру](/notebooks/lesson%208%20-%20Open%20Close%20comments%20VKontakte%20timer.ipynb)